# 텍스트 마이닝을 활용한 영화 평점 감성 분석

## 1) 영화 평점 데이터 크롤링

In [25]:
import requests
from bs4 import BeautifulSoup
from urllib import parse

base_url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=208077&target=after&page=2"
url = base_url.format(1)
res = requests.get(url)

soup = BeautifulSoup(res.text)
tds = soup.select('table.list_netizen > tbody > tr > td.title')

for td in tds:
    movie_title = td.select_one('a.movie').text.strip()
    score = td.select_one('div.list_netizen_score > em').text.strip()
    comment = td.select_one('br').next_sibling.strip()
    print(movie_title, score, comment, sep=' :: ')
    print('-------------------------------------------')

스파이더맨: 노 웨이 홈 :: 7 :: 쏘쏘~ 마블 취향이 아닌지라..
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 스파이더맨 전 시리즈를 안봤음에도 재미있었어요.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 8 :: 액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 보면 볼 수록 과거의 아련함이 묻어나온다.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 2시간 영화를 봤는데, 본게 없다 .다시 봐야해
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 앤드류 갓필드...
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 6 :: 스토리 박살.  로맨스물. 평점엔 알바들만 득실. 소니 가 소니햇다.
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 7 :: 스파이더맨을 볼 수 밖에 없는 상영관이네요 ㅠㅠ
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 그냥 감동입니다 꼭보세요
-------------------------------------------
스파이더맨: 노 웨이 홈 :: 10 :: 10대의 나와 20대의 나..그리고 지금의 내가 만난듯한 기분
-------------------------------------------


In [33]:
import time
import random

comment_list=[] # 저장 할 리스트
for page in range(1, 101):
    url = base_url.format(page)
    res = requests.get(url)
    
    soup = BeautifulSoup(res.text, 'lxml')
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    
    for td in tds:
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        comment_list.append((score, comment)) # 리스트에 추가
    delay = round(random.uniform(0.2, 1.2), 2) # 크롤링을 위해 페이지 전환을 빠르게 하게되면 네이버에서 막을 수 있기에 delay를 걸어준다.
    time.sleep(delay)
    

In [32]:
print(comment_list)

[('7', '쏘쏘~ 마블 취향이 아닌지라..'), ('10', '스파이더맨 전 시리즈를 안봤음에도 재미있었어요.'), ('8', '액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요'), ('10', '보면 볼 수록 과거의 아련함이 묻어나온다.'), ('10', '2시간 영화를 봤는데, 본게 없다 .다시 봐야해'), ('10', '앤드류 갓필드...'), ('6', '스토리 박살.  로맨스물. 평점엔 알바들만 득실. 소니 가 소니햇다.'), ('7', '스파이더맨을 볼 수 밖에 없는 상영관이네요 ㅠㅠ'), ('10', '그냥 감동입니다 꼭보세요'), ('10', '10대의 나와 20대의 나..그리고 지금의 내가 만난듯한 기분'), ('7', '쏘쏘~ 마블 취향이 아닌지라..'), ('10', '스파이더맨 전 시리즈를 안봤음에도 재미있었어요.'), ('8', '액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요'), ('10', '보면 볼 수록 과거의 아련함이 묻어나온다.'), ('10', '2시간 영화를 봤는데, 본게 없다 .다시 봐야해'), ('10', '앤드류 갓필드...'), ('6', '스토리 박살.  로맨스물. 평점엔 알바들만 득실. 소니 가 소니햇다.'), ('7', '스파이더맨을 볼 수 밖에 없는 상영관이네요 ㅠㅠ'), ('10', '그냥 감동입니다 꼭보세요'), ('10', '10대의 나와 20대의 나..그리고 지금의 내가 만난듯한 기분'), ('7', '쏘쏘~ 마블 취향이 아닌지라..'), ('10', '스파이더맨 전 시리즈를 안봤음에도 재미있었어요.'), ('8', '액션은 부족했지만 스파이더맨팬에게는 선물 같은 영화였어요'), ('10', '보면 볼 수록 과거의 아련함이 묻어나온다.'), ('10', '2시간 영화를 봤는데, 본게 없다 .다시 봐야해'), ('10', '앤드류 갓필드...'), ('6', '스토리 박살.  로맨스물. 평점엔 알바들만 득실. 소니 가 소니햇다.'), ('7', '스파이더맨을 볼 수 밖에 없는 상영관

In [29]:
import pandas as pd
df = pd.DataFrame(comment_list,columns=['rate','comment'])
df.to_csv('spider_naver_comment.csv', encoding='utf-8', index = False)

## 2) 텍스트 전처리를 위한 데이터 셋 살펴보기

In [30]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv("https://raw.githubusercontent.com/nexussord/HUFS_MLP/main/spider_naver_comment.csv")

In [19]:
df.head()

,평점,댓글
0,10,말이 필요없다.개연성 오마주 팬서비스 모든걸 잘 버무린 최고의 영화..! 재미없고 ...
1,10,어렸을때 추억이 새록새록 앞으로가 기대됩니다.
2,10,"스토리, 개연성 다 필요없음.앤드류 가필드가 톰 홀랜드의 MJ 를 무사히 구출하는 ..."
3,10,.나하나하우투아가아고
4,10,지림..진짜재밋게봄 또보러갈구


In [20]:
df.shape

(1000, 2)

In [21]:
df.isnull().sum() # 데이터 무결성 확인

평점    0
댓글    0
dtype: int64

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   평점      1000 non-null   int64 
 1   댓글      1000 non-null   object
dtypes: int64(1), object(1)
memory usage: 15.8+ KB


In [24]:
df['댓'][0]

'말이 필요없다.개연성 오마주 팬서비스 모든걸 잘 버무린 최고의 영화..! 재미없고 이상하다고 한 사람들은 아마 대부분 이전 스파이더맨 영화를 제대로 보지 않았거나 히스토리를 잘 몰랐던 사람인거 같다.'

In [34]:
len(df['text'].values.sum())

KeyError: 'text'